## Story Generator Colab Server
https://colab.research.google.com/drive/1uNygzDR4hISwLOgmDS31hRHfr6KAF7Ib?usp=sharing

In [ ]:
!pip install -q transformers==4.35.2 accelerate bitsandbytes==0.41.1 huggingface_hub
!pip install -q Flask==2.3.2 Pillow==9.5.0 requests==2.30.0 flask-ngrok Flask-CORS==3.0.10 pyngrok
!pip install Flask-Cors

from huggingface_hub import login
login("your_huggingface_token")  # Replace with your token or use secrets

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from IPython.display import clear_output
clear_output(wait=False)

print("Dependencies installed successfully!")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from pyngrok import ngrok
import base64
from PIL import Image
import io

# Configure ngrok
ngrok.set_auth_token('2oHUyMyGNJuD34GO6NdGJd8KAxd_3TyYzUGLMA9DvgUopNRw3')

app = Flask(__name__)
CORS(app)

# Global variables for models
story_tokenizer = None
story_model = None
device = None

def load_models():
    global story_tokenizer, story_model, device
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
        
        model_name = "facebook/opt-1.3b"  # Using this as primary model instead
        
        story_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_8bit=True
        )
        
        story_tokenizer = AutoTokenizer.from_pretrained(model_name)
        print(f"Successfully loaded {model_name}")
        
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        raise

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.json
        if not data:
            return jsonify({'success': False, 'error': 'No data received'}), 400
        
        genre = data.get('genre', 'fantasy')
        length = data.get('length', 'short')
        
        # Craft prompt
        prompt = f"Write a {genre.lower()} story with the following elements:\n" \
                 f"- Engaging plot and characters\n" \
                 f"- {length} length\n" \
                 f"- Clear beginning, middle, and end\n" \
                 f"- Age-appropriate content\n" \
                 f"- Emotional resonance\n" \
                 f"\n" \
                 f"Once upon a time, in a land far away, there lived a hero named [Hero's Name]. " \
                 f"[Hero's Name] was known for [Hero's Trait] and had a mission to [Hero's Mission]. " \
                 f"One day, [Hero's Name] encountered a challenge: [Challenge]. " \
                 f"With courage and determination, [Hero's Name] set out to overcome this challenge. " \
                 f"Along the way, [Hero's Name] met [Supporting Character], who helped in [Supporting Character's Role]. " \
                 f"Together, they faced many obstacles, including [Obstacle 1], [Obstacle 2], and [Obstacle 3]. " \
                 f"In the end, [Hero's Name] achieved [Outcome], bringing peace and happiness to the land. " \
                 f"And so, the story of [Hero's Name] became a legend, inspiring many for generations to come."
        
        # Generation parameters
        max_new_tokens = {'short': 500, 'medium': 1000, 'long': 1500}.get(length.lower(), 600)
        
        # Generate story
        inputs = story_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
        output_sequences = story_model.generate(
            input_ids=inputs['input_ids'],
            max_new_tokens=max_new_tokens,
            temperature=0.7,  # Lower temperature for more coherent text
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=story_tokenizer.eos_token_id
        )

        # Process output
        story = story_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
        final_story = story.replace(prompt, '').strip()
        
        if not final_story:
            return jsonify({'success': False, 'error': 'Failed to generate story'}), 500
            
        return jsonify({'success': True, 'story': final_story})

    except Exception as e:
        print(f"Server error: {str(e)}")
        return jsonify({'success': False, 'error': f'Server error: {str(e)}'}), 500

if __name__ == '__main__':
    print("Loading AI models...")
    load_models()
    print("Models loaded successfully!")
    
    public_url = ngrok.connect(5000)
    api_url = f"{public_url.public_url}/generate_story"
    print(f'Server running at: {api_url}')
    
    app.run(port=5000)